In [49]:
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm

In [50]:
fpath = "/content/drive/MyDrive/論文資料/Processed Data/WheatQuantity.xlsx"
df = pd.read_excel(fpath)

In [51]:
df

,Area,Element,Item,Year,Unit,Value,critical oni,return,enso^2,return(-1),Elnino,Lanina
0,United States of America,Quantity,Wheat,1961,kg/ha,33539008,0.0875,NaN,0.007656,NaN,0,0
1,United States of America,Quantity,Wheat,1962,kg/ha,29718000,-0.2375,-0.113927,0.056406,NaN,0,0
2,United States of America,Quantity,Wheat,1963,kg/ha,31211888,-0.0875,0.050269,0.007656,-0.113927,0,0
3,United States of America,Quantity,Wheat,1964,kg/ha,34928000,0.6125,0.119061,0.375156,0.050269,1,0
4,United States of America,Quantity,Wheat,1965,kg/ha,35805008,-0.3375,0.025109,0.113906,0.119061,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58,United States of America,Quantity,Wheat,2019,kg/ha,52580890,0.7250,0.024858,0.525625,0.082857,1,0
59,United States of America,Quantity,Wheat,2020,kg/ha,49751180,0.3500,-0.053816,0.122500,0.024858,0,0
60,United States of America,Quantity,Wheat,2021,kg/ha,44803690,-0.9500,-0.099445,0.902500,-0.053816,0,1
61,United States of America,Quantity,Wheat,2022,kg/ha,44897830,-0.9750,0.002101,0.950625,-0.099445,0,1


In [52]:
y_col   = "return"        # ⇦ 這裡換成你的 dependent 變數
oni_col = "critical oni"  # ⇦ 這裡換成你的 ONI 欄

In [53]:
data = (
    df[[y_col, oni_col]]
    .dropna()
    .rename(columns={y_col: "y", oni_col: "oni"})
)

In [54]:
data

,y,oni
1,-0.113927,-0.2375
2,0.050269,-0.0875
3,0.119061,0.6125
4,0.025109,-0.3375
5,-0.008132,1.3000
...,...,...
58,0.024858,0.7250
59,-0.053816,0.3500
60,-0.099445,-0.9500
61,0.002101,-0.9750


In [55]:
# === 2. 建立門檻候選 grid ==============================================
# 10% ~ 90% 中先取 50 個點，再保留唯一值
q_lo, q_hi = 0.10, 0.90
n_grid     = 50
grid_vals  = np.unique(
    np.quantile(data["oni"], np.linspace(q_lo, q_hi, n_grid))
)

min_obs = int(0.10 * len(data))   # 每段至少佔 10% 樣本
min_gap = 0.1             # 門檻至少相隔 0.1 °C

In [56]:
# === 3. 定義一個計分函數 (這裡用 SSE) ================================
def piecewise_sse(df_part):
    X = sm.add_constant(df_part["oni"])
    model = sm.OLS(df_part["y"], X).fit()
    return (model.resid ** 2).sum()

results = []
for c1, c2 in itertools.combinations(grid_vals, 2):
    if c2 - c1 < min_gap:
        continue

    reg1 = data[data["oni"] <= c1]
    reg2 = data[(data["oni"] > c1) & (data["oni"] <= c2)]
    reg3 = data[data["oni"] > c2]

    # 檢查子樣本數
    if min(len(reg1), len(reg2), len(reg3)) < min_obs:
        continue

    sse = (
        piecewise_sse(reg1)
        + piecewise_sse(reg2)
        + piecewise_sse(reg3)
    )
    results.append({"c1": c1, "c2": c2, "sse": sse})

In [57]:
# === 4. 找出最佳門檻 ==================================================
res_df = pd.DataFrame(results).sort_values("sse", ascending=True)
best = res_df.iloc[0]

print(f"最佳門檻（SSE 最小）: c1 = {best.c1:.3f}, c2 = {best.c2:.3f}")
print(f"對應 SSE = {best.sse:.2f}")

# 如果想看前 10 名：
print(res_df.head(10))

最佳門檻（SSE 最小）: c1 = 0.511, c2 = 0.673
對應 SSE = 0.90
           c1        c2       sse
932  0.511020  0.672908  0.900036
897  0.349515  0.672908  0.916544
797  0.049107  0.446173  0.937118
798  0.049107  0.485434  0.941483
795  0.049107  0.350000  0.960033
796  0.049107  0.373827  0.960033
802  0.049107  0.608929  0.961343
793  0.049107  0.336199  0.961994
365 -0.609337  0.136582  0.963540
927  0.485434  0.672908  0.964457


In [58]:
# === 5. 用最佳門檻做完整分段模型 (可選) ================================
# 建立 regime dummy
data["reg_L"] = (data["oni"] <= best.c1).astype(int)
data["reg_M"] = ((data["oni"] > best.c1) & (data["oni"] <= best.c2)).astype(int)
data["reg_H"] = (data["oni"] > best.c2).astype(int)

# 建立交互項：ONI × regime
for r in ["L", "M", "H"]:
    data[f"oni_{r}"] = data["oni"] * data[f"reg_{r}"]

X_full = sm.add_constant(data[["reg_L", "reg_M", "oni_L", "oni_M", "oni_H"]])
model_full = sm.OLS(data["y"], X_full).fit()
print(model_full.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     2.554
Date:                Wed, 04 Jun 2025   Prob (F-statistic):             0.0376
Time:                        05:57:25   Log-Likelihood:                 43.232
No. Observations:                  62   AIC:                            -74.46
Df Residuals:                      56   BIC:                            -61.70
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0339      0.119     -0.284      0.7

In [59]:
data.to_excel("WheatGridSearch.xlsx")

In [60]:
fpath2 = "/content/drive/MyDrive/論文資料/Processed Data/CornQuantity.xlsx"
df = pd.read_excel(fpath2)

In [61]:
df

,Area,Element,Item,Year,Unit,Value,critical oni,return,enso^2,return(-1),Elnino,Lanian
0,United States of America,Yield,Maize (corn),1961,kg/ha,91388000,0.12,NaN,0.0144,NaN,0,1
1,United States of America,Yield,Maize (corn),1962,kg/ha,91604000,-0.18,0.002364,0.0324,NaN,0,1
2,United States of America,Yield,Maize (corn),1963,kg/ha,102093008,0.62,0.114504,0.3844,0.002364,1,1
3,United States of America,Yield,Maize (corn),1964,kg/ha,88504000,-0.56,-0.133104,0.3136,0.114504,0,0
4,United States of America,Yield,Maize (corn),1965,kg/ha,104216928,0.84,0.177539,0.7056,-0.133104,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
58,United States of America,Yield,Maize (corn),2019,kg/ha,345962110,0.42,-0.050239,0.1764,-0.018415,0,1
59,United States of America,Yield,Maize (corn),2020,kg/ha,358447310,-0.24,0.036088,0.0576,-0.050239,0,1
60,United States of America,Yield,Maize (corn),2021,kg/ha,381469380,-0.50,0.064227,0.2500,0.036088,0,0
61,United States of America,Yield,Maize (corn),2022,kg/ha,346739460,-0.94,-0.091042,0.8836,0.064227,0,0


In [62]:
data = (
    df[[y_col, oni_col]]
    .dropna()
    .rename(columns={y_col: "y", oni_col: "oni"})
)

In [63]:
data

,y,oni
1,0.002364,-0.18
2,0.114504,0.62
3,-0.133104,-0.56
4,0.177539,0.84
5,0.015779,0.32
...,...,...
58,-0.050239,0.42
59,0.036088,-0.24
60,0.064227,-0.50
61,-0.091042,-0.94


In [64]:
# === 2. 建立門檻候選 grid ==============================================
# 10% ~ 90% 中先取 50 個點，再保留唯一值
q_lo, q_hi = 0.10, 0.90
n_grid     = 50
grid_vals  = np.unique(
    np.quantile(data["oni"], np.linspace(q_lo, q_hi, n_grid))
)

min_obs = int(0.10 * len(data))   # 每段至少佔 10% 樣本
min_gap = 0.1             # 門檻至少相隔 0.1 °C

In [65]:
results = []
for c1, c2 in itertools.combinations(grid_vals, 2):
    if c2 - c1 < min_gap:
        continue

    reg1 = data[data["oni"] <= c1]
    reg2 = data[(data["oni"] > c1) & (data["oni"] <= c2)]
    reg3 = data[data["oni"] > c2]

    # 檢查子樣本數
    if min(len(reg1), len(reg2), len(reg3)) < min_obs:
        continue

    sse = (
        piecewise_sse(reg1)
        + piecewise_sse(reg2)
        + piecewise_sse(reg3)
    )
    results.append({"c1": c1, "c2": c2, "sse": sse})

In [66]:
# === 4. 找出最佳門檻 ==================================================
res_df = pd.DataFrame(results).sort_values("sse", ascending=True)
best = res_df.iloc[0]

print(f"最佳門檻（SSE 最小）: c1 = {best.c1:.3f}, c2 = {best.c2:.3f}")
print(f"對應 SSE = {best.sse:.2f}")

# 如果想看前 10 名：
print(res_df.head(10))

最佳門檻（SSE 最小）: c1 = -0.640, c2 = -0.360
對應 SSE = 1.85
           c1        c2       sse
45  -0.640000 -0.360000  1.845995
46  -0.640000 -0.357796  1.845995
86  -0.636327 -0.357796  1.845995
85  -0.636327 -0.360000  1.845995
124 -0.596490 -0.360000  1.872056
125 -0.596490 -0.357796  1.872056
270 -0.498653 -0.300000  1.927917
271 -0.498653 -0.297673  1.927917
272 -0.498653 -0.237918  1.928375
273 -0.498653 -0.180000  1.937345


In [67]:
# === 5. 用最佳門檻做完整分段模型 (可選) ================================
# 建立 regime dummy
data["reg_L"] = (data["oni"] <= best.c1).astype(int)
data["reg_M"] = ((data["oni"] > best.c1) & (data["oni"] <= best.c2)).astype(int)
data["reg_H"] = (data["oni"] > best.c2).astype(int)

# 建立交互項：ONI × regime
for r in ["L", "M", "H"]:
    data[f"oni_{r}"] = data["oni"] * data[f"reg_{r}"]

X_full = sm.add_constant(data[["reg_L", "reg_M", "oni_L", "oni_M", "oni_H"]])
model_full = sm.OLS(data["y"], X_full).fit()
print(model_full.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     4.958
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           0.000792
Time:                        05:57:33   Log-Likelihood:                 20.963
No. Observations:                  62   AIC:                            -29.93
Df Residuals:                      56   BIC:                            -17.16
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0318      0.033      0.971      0.3

In [68]:
fpath3 = "/content/drive/MyDrive/論文資料/Processed Data/SoybeanQuantity.xlsx"
df = pd.read_excel(fpath3)

In [69]:
df

,Area,Element,Item,Year,Unit,Value,critical oni,return,enso^2,return(-1),ElNino,LaNina
0,United States of America,Yield,Soya beans,1961,kg/ha,18468000,0.04,NaN,0.0016,NaN,0,0
1,United States of America,Yield,Soya beans,1962,kg/ha,18213008,-0.14,-0.013807,0.0196,NaN,0,0
2,United States of America,Yield,Soya beans,1963,kg/ha,19028000,0.80,0.044748,0.6400,-0.013807,1,1
3,United States of America,Yield,Soya beans,1964,kg/ha,19076000,-0.66,0.002523,0.4356,0.044748,0,0
4,United States of America,Yield,Soya beans,1965,kg/ha,23014000,1.18,0.206437,1.3924,0.002523,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
58,United States of America,Yield,Soya beans,2019,kg/ha,96667090,0.32,-0.197880,0.1024,0.003744,0,0
59,United States of America,Yield,Soya beans,2020,kg/ha,114748940,-0.46,0.187053,0.2116,-0.197880,0,0
60,United States of America,Yield,Soya beans,2021,kg/ha,121503600,-0.50,0.058865,0.2500,0.187053,0,0
61,United States of America,Yield,Soya beans,2022,kg/ha,116220720,-0.92,-0.043479,0.8464,0.058865,0,0


In [70]:
data = (
    df[[y_col, oni_col]]
    .dropna()
    .rename(columns={y_col: "y", oni_col: "oni"})
)

In [71]:
data

,y,oni
1,-0.013807,-0.14
2,0.044748,0.80
3,0.002523,-0.66
4,0.206437,1.18
5,0.098027,0.16
...,...,...
58,-0.197880,0.32
59,0.187053,-0.46
60,0.058865,-0.50
61,-0.043479,-0.92


In [72]:
# === 2. 建立門檻候選 grid ==============================================
# 10% ~ 90% 中先取 50 個點，再保留唯一值
q_lo, q_hi = 0.10, 0.90
n_grid     = 50
grid_vals  = np.unique(
    np.quantile(data["oni"], np.linspace(q_lo, q_hi, n_grid))
)

min_obs = int(0.10 * len(data))   # 每段至少佔 10% 樣本
min_gap = 0.1             # 門檻至少相隔 0.1 °C

In [73]:
results = []
for c1, c2 in itertools.combinations(grid_vals, 2):
    if c2 - c1 < min_gap:
        continue

    reg1 = data[data["oni"] <= c1]
    reg2 = data[(data["oni"] > c1) & (data["oni"] <= c2)]
    reg3 = data[data["oni"] > c2]

    # 檢查子樣本數
    if min(len(reg1), len(reg2), len(reg3)) < min_obs:
        continue

    sse = (
        piecewise_sse(reg1)
        + piecewise_sse(reg2)
        + piecewise_sse(reg3)
    )
    results.append({"c1": c1, "c2": c2, "sse": sse})

In [74]:
# === 4. 找出最佳門檻 ==================================================
res_df = pd.DataFrame(results).sort_values("sse", ascending=True)
best = res_df.iloc[0]

print(f"最佳門檻（SSE 最小）: c1 = {best.c1:.3f}, c2 = {best.c2:.3f}")
print(f"對應 SSE = {best.sse:.2f}")

# 如果想看前 10 名：
print(res_df.head(10))

最佳門檻（SSE 最小）: c1 = 0.179, c2 = 0.359
對應 SSE = 0.81
           c1        c2       sse
826  0.179388  0.358898  0.814355
814  0.160000  0.358898  0.814355
773  0.139796  0.338980  0.822999
801  0.159551  0.338980  0.829530
787  0.140000  0.338980  0.829530
757  0.119510  0.338980  0.833297
821  0.160000  0.533306  0.840733
833  0.179388  0.533306  0.840733
225 -0.578408  0.358898  0.846309
188 -0.580000  0.358898  0.846309


In [75]:
# === 5. 用最佳門檻做完整分段模型 (可選) ================================
# 建立 regime dummy
data["reg_L"] = (data["oni"] <= best.c1).astype(int)
data["reg_M"] = ((data["oni"] > best.c1) & (data["oni"] <= best.c2)).astype(int)
data["reg_H"] = (data["oni"] > best.c2).astype(int)

# 建立交互項：ONI × regime
for r in ["L", "M", "H"]:
    data[f"oni_{r}"] = data["oni"] * data[f"reg_{r}"]

X_full = sm.add_constant(data[["reg_L", "reg_M", "oni_L", "oni_M", "oni_H"]])
model_full = sm.OLS(data["y"], X_full).fit()
print(model_full.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.275
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     4.242
Date:                Wed, 04 Jun 2025   Prob (F-statistic):            0.00243
Time:                        05:57:40   Log-Likelihood:                 46.333
No. Observations:                  62   AIC:                            -80.67
Df Residuals:                      56   BIC:                            -67.90
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1710      0.063      2.703      0.0